## Knowledge Bases - Reranker

---

Bedrock provides access to reranking models through the Rerank API which will help to further improve the accuracy and relevance of RAG applications. This notebook provides sample code for using reranker models to improve retrieval quality. 


---

#### Notebook Walkthrough

This notebook explores building a Q&A application using Bedrock Knowledgebase RetrieveAndGenerate API with a reranker model. We'll compare results with and without reranking, and evaluate responses using metrics including faithfulness, relevancy, recall, precision, and correctness.

#### Pre-requisite: 

This notebook generates question, answer, context from PDF Files and loads to Amazon S3.Steps:

* Pre-requisite:
  1. For S3 , create s3 bucket (if not exists)
  2. Provide access to the Foundation Models in Amazon Bedrock

#### Setup:

Before being able to answer the questions, the documents must be processed and stored in a knowledge base. For this notebook, we use a synthetic dataset for 10K financial reports to create the Knowledge Bases for Amazon Bedrock.
  1. Upload your documents (data source) to Amazon S3 bucket.
  2. Create Amazon Bedrock Knowlegebase using [01_create_ingest_documents_test_kb_multi_ds.ipynb](https://github.com/aws-samples/amazon-bedrock-samples/blob/d730ca87c75a3801670677a14cef1ff6a298410c/rag/knowledge-bases/features-examples/01-rag-concepts/01_create_ingest_documents_test_kb_multi_ds.ipynb)
  3. Note the Knowledge Base ID
  4. **Internal beta testing** Follow https://quip-amazon.com/q8vqAQBC6DyZ/Bedrock-Reranker-Usage-Instructions to setup access to reranker and boto3 (TODO: remove when publishing)

---

In [ ]:
%pip install -q ragas langchain langchain_aws langchain_community

Install beta Boto3 to test the feature out

TODO: remove the cell when publishing

```
%pip install botocore-1.35.65-py3-none-any.whl boto3-1.35.65-py3-none-any.whl
```

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%store -r kb_id
kb_id = "YWU2HLLTET" # Replace with your knowledge base id here.

# kb_id = "<<knowledge_base_id>>" # Replace with your knowledge base id here.

no stored variable or alias kb_id


In [3]:
import boto3
import pprint
from botocore.client import Config
from langchain.llms.bedrock import Bedrock
from langchain_aws import ChatBedrock
from langchain_aws import BedrockEmbeddings
import time
import json

print(boto3.__version__)

1.35.65


In [4]:
bedrock_client = boto3.client('bedrock-runtime')

# TODO remove the gamma endpoint when publishing
bedrock_agent_client = boto3.client(
    "bedrock-agent-runtime",
    endpoint_url="https://gamma.us-east-1.runtime.bedrock-agent.aws.dev/",
    region_name="us-east-1",
)

TEXT_GENERATION_MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
EVALUATION_MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v2:0"


# Reranker model: there are two reranker models available at launch
AMAZON_RERANKER_MODEL = "arn:aws:bedrock:us-east-1::foundation-model/amazon.rerank-v1:0"
COHERE_RERANKER_MODEL = "arn:aws:bedrock:us-east-1::foundation-model/cohere.rerank-v3-5:0"


llm_for_evaluation = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", client=bedrock_client)
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0", client=bedrock_client)

---

#### Use reranker with Retrieve and RetrieveAndGenerate

Observe:
1. Retrieval relevancy score and reranked score.
2. Latency add-on

---

In [ ]:
def get_retrieval_config(reranker_model=None):
    retrieval_config = None
    if reranker_model:
        retrieval_config = {
            "vectorSearchConfiguration": {
                "numberOfResults": 30,
                "rerankingConfiguration": {
                    "type": "BEDROCK_RERANKING_MODEL",
                    "bedrockRerankingConfiguration": {
                        "modelConfiguration": {
                        "modelArn": reranker_model
                        },
                        "numberOfRerankedResults": 3
                    }
                }
            }
        }
    else:
        retrieval_config = {
            "vectorSearchConfiguration": {
                "numberOfResults": 3
            }
        }
    return retrieval_config

In [6]:
def retrieve_and_generate(query, retrieval_config):
    start = time.time()
    response = bedrock_agent_client.retrieve_and_generate(
        input={
                'text': query
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,
                    'modelArn': TEXT_GENERATION_MODEL_ID,
                    'retrievalConfiguration': retrieval_config,
                },
            }
        )
    time_spent = time.time() - start
    print(f"[Response]\n{response['output']['text']}\n")
    print(f"[Invocation time]\n{time_spent}\n")

    return response

In [7]:
def retrieve_docs(query, retrieval_config):
    start = time.time()
    response = bedrock_agent_client.retrieve(
        retrievalQuery={
            'text': query
        },
        knowledgeBaseId=kb_id,
        retrievalConfiguration=retrieval_config
    )
    time_spent = time.time() - start
    print(f"[Retrieved docs]\n{json.dumps(response['retrievalResults'], indent=2)}\n")
    print(f"[Invocation time]\n{time_spent}\n")

    return response

---

**Without reranker:**  retrieve 3 chunks

---

In [8]:
query = "Provide a list of few risks for Octank financial in numbered list without description."

# without reranker
retrieval_config = get_retrieval_config(reranker_model=None)

retrieve_docs(query, retrieval_config)
retrieve_and_generate(query, retrieval_config)

[Retrieved docs]
[
  {
    "content": {
      "text": "However, the company still faces significant risks if any of its key suppliers were to fail.     ### Market Risks     Octank Financial's financial performance is subject to various market risks, including changes in commodity prices, foreign exchange rates, and equity prices. For example, a significant decline in the price of oil could have a material adverse effect on Octank Financial's revenue and profitability. Similarly, changes in foreign exchange rates could impact the company's cost of goods sold and profitability.     ### Table: Market Risks     | Risk Factor | Description | Potential Impact | | --- | --- | --- | | Commodity Prices | Changes in commodity prices, such as oil, could impact Octank Financial's revenue and profitability. | Material adverse effect on revenue and profitability. | | Foreign Exchange Rates | Changes in foreign exchange rates could impact the company's cost of goods sold and profitability. | Material

{'ResponseMetadata': {'RequestId': 'cc7f8779-007f-4db3-a7ff-cf1f97c5e0c4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 25 Nov 2024 14:03:30 GMT',
   'content-type': 'application/json',
   'content-length': '6393',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'cc7f8779-007f-4db3-a7ff-cf1f97c5e0c4'},
  'RetryAttempts': 0},
 'sessionId': '256f4f1b-deae-40e8-a3fb-908fe4153c52',
 'output': {'text': 'The key risks for Octank Financial include:\n\n1. Liquidity risk\n2. Operational risk\n3. Regulatory risk\n4. Strategic risk\n5. Market risks (commodity prices, foreign exchange rates, equity prices)'},
 'citations': [{'generatedResponsePart': {'textResponsePart': {'text': 'The key risks for Octank Financial include:\n\n1. Liquidity risk\n2. Operational risk\n3. Regulatory risk\n4. Strategic risk\n5. Market risks (commodity prices, foreign exchange rates, equity prices)',
     'span': {'start': 0, 'end': 192}}},
   'retrievedReferences': [{'content': {'text': 'Our liquidity

---

**With reranker:** retrieve 50 chunks and get 3 reranked chunks back


Notice that the scores in the response are reranked scores now.

---

In [9]:
# With Amazon reranker model
reranker_retrieval_config = get_retrieval_config(reranker_model=AMAZON_RERANKER_MODEL)

retrieve_docs(query, reranker_retrieval_config)
# TODO reranker in gamma env throttles a lot. remove the time.sleep when publishing the code
time.sleep(60)
retrieve_and_generate(query, reranker_retrieval_config)

[Retrieved docs]
[
  {
    "content": {
      "text": "However, the company still faces significant risks if any of its key suppliers were to fail.     ### Market Risks     Octank Financial's financial performance is subject to various market risks, including changes in commodity prices, foreign exchange rates, and equity prices. For example, a significant decline in the price of oil could have a material adverse effect on Octank Financial's revenue and profitability. Similarly, changes in foreign exchange rates could impact the company's cost of goods sold and profitability.     ### Table: Market Risks     | Risk Factor | Description | Potential Impact | | --- | --- | --- | | Commodity Prices | Changes in commodity prices, such as oil, could impact Octank Financial's revenue and profitability. | Material adverse effect on revenue and profitability. | | Foreign Exchange Rates | Changes in foreign exchange rates could impact the company's cost of goods sold and profitability. | Material

{'ResponseMetadata': {'RequestId': 'ae70abd4-badc-444e-9f48-f2beb7951aa8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 25 Nov 2024 14:04:33 GMT',
   'content-type': 'application/json',
   'content-length': '5859',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ae70abd4-badc-444e-9f48-f2beb7951aa8'},
  'RetryAttempts': 0},
 'sessionId': '9c4c9c88-eed0-4647-9d97-39acca06c196',
 'output': {'text': '1. Compliance risks\n2. Cybersecurity risks\n3. Dependence on key suppliers\n4. Market risks (commodity prices, foreign exchange rates, equity prices)'},
 'citations': [{'generatedResponsePart': {'textResponsePart': {'text': '1. Compliance risks\n2. Cybersecurity risks\n3. Dependence on key suppliers\n4. Market risks (commodity prices, foreign exchange rates, equity prices)',
     'span': {'start': 0, 'end': 146}}},
   'retrievedReferences': [{'content': {'text': "For example, PersonB, Octank Financial's Compliance Officer, has been working diligently to ensure that the com

---

#### Use RAGAS to evaluate RAG

As RAGAS aims to be a reference-free evaluation framework, the required preparations of the evaluation dataset are minimal. You will need to prepare question and ground_truths pairs from which you can prepare the remaining information through inference as shown below. If you are not interested in the context_recall metric, you don’t need to provide the ground_truths information. In this case, all you need to prepare are the questions.

---

In [10]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    context_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_correctness,
)

#specify the metrics here
metrics = [
    context_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_correctness,
]


questions = [
    "What was the primary reason for the increase in net cash provided by operating activities for Octank Financial in 2021?",
    "In which year did Octank Financial have the highest net cash used in investing activities, and what was the primary reason for this?",
    "What was the primary source of cash inflows from financing activities for Octank Financial in 2021?",
    "Based on the information provided, what can you infer about Octank Financial's overall financial health and growth prospects?"
]
ground_truths = [
    "The increase in net cash provided by operating activities was primarily due to an increase in net income and favorable changes in operating assets and liabilities.",
    "Octank Financial had the highest net cash used in investing activities in 2021, at $360 million. The primary reason for this was an increase in purchases of property, plant, and equipment and marketable securities",
    "The primary source of cash inflows from financing activities for Octank Financial in 2021 was an increase in proceeds from the issuance of common stock and long-term debt.",
    "Based on the information provided, Octank Financial appears to be in a healthy financial position and has good growth prospects. The company has consistently increased its net cash provided by operating activities, indicating strong profitability and efficient management of working capital. Additionally, Octank Financial has been investing in long-term assets, such as property, plant, and equipment, and marketable securities, which suggests plans for future growth and expansion. The company has also been able to finance its growth through the issuance of common stock and long-term debt, indicating confidence from investors and lenders. Overall, Octank Financial's steady increase in cash and cash equivalents over the past three years provides a strong foundation for future growth and investment opportunities."
]

In [11]:
def prepare_eval_dataset(retrieval_config, questions, ground_truths):

    answers = []
    contexts = []
    for query in questions:
        response = retrieve_and_generate(query, retrieval_config)
        answers.append(response["output"]["text"])
        for citation in response["citations"]:
            context_group = [
                ref["content"]["text"]
                for ref in citation["retrievedReferences"]
                if "content" in ref and "text" in ref["content"]
            ]
        contexts.append(context_group)
        # the gamma env doesn't allow high invocation rate. 
        # thow a sleep of 1 min to avoid it
        # TODO remove when publishing
        time.sleep(60)

    # To dict
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truth": ground_truths
    }

    # Convert dict to dataset
    dataset = Dataset.from_dict(data)
    return dataset

In [12]:
without_reranker_dataset = prepare_eval_dataset(retrieval_config, questions, ground_truths)

[Response]
The primary reason for the increase in net cash provided by operating activities for Octank Financial in 2021 was an increase in net income and favorable changes in operating assets and liabilities.

[Invocation time]
1.3246502876281738

[Response]
According to the search results, Octank Financial had the highest net cash used in investing activities in 2021, at $360 million. The primary reason for this was an increase in purchases of property, plant, and equipment and marketable securities.

[Invocation time]
1.3529317378997803

[Response]
According to the search results, the primary source of cash inflows from financing activities for Octank Financial in 2021 was proceeds from the issuance of long-term debt, which amounted to $500 million.

[Invocation time]
1.5769374370574951

[Response]
Based on the information provided, Octank Financial appears to be in strong financial health and have good growth prospects: - The company reported robust revenue growth of 15% in 2021, i

In [13]:
without_reranker_result = evaluate(
    dataset=without_reranker_dataset,
    metrics=metrics,
    llm=llm_for_evaluation,
    embeddings=bedrock_embeddings,
)

without_reranker_result_df = without_reranker_result.to_pandas()

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

In [14]:
without_reranker_result_df

,question,answer,contexts,ground_truth,context_relevancy,context_recall,context_precision,context_entity_recall,answer_correctness
0,What was the primary reason for the increase i...,The primary reason for the increase in net cas...,[. The statement is divided into three main pa...,The increase in net cash provided by operating...,0.222222,1.0,1.0,0.000000,0.919630
1,In which year did Octank Financial have the hi...,"According to the search results, Octank Financ...",[. The statement is divided into three main pa...,Octank Financial had the highest net cash used...,0.222222,1.0,1.0,0.400000,0.745553
2,What was the primary source of cash inflows fr...,"According to the search results, the primary s...",[** | | | | | Proceeds from issuance of common...,The primary source of cash inflows from financ...,0.750000,1.0,1.0,0.250000,0.606073
3,"Based on the information provided, what can yo...","Based on the information provided, Octank Fina...","[At Octank Financial, we are committed to deli...","Based on the information provided, Octank Fina...",0.125000,0.4,0.0,0.166667,0.366631


In [15]:
with_reranker_dataset = prepare_eval_dataset(reranker_retrieval_config, questions, ground_truths)

[Response]
The primary reason for the increase in net cash provided by operating activities for Octank Financial in 2021 was an increase in net income and favorable changes in operating assets and liabilities.

[Invocation time]
2.3456826210021973

[Response]
According to the search results, Octank Financial had the highest net cash used in investing activities in 2021, at $360 million. The primary reason for this was an increase in purchases of property, plant, and equipment and marketable securities.

[Invocation time]
2.249284505844116

[Response]
According to the search results, the primary source of cash inflows from financing activities for Octank Financial in 2021 was an increase in proceeds from issuance of common stock and long-term debt.

[Invocation time]
2.2653369903564453

[Response]
Based on the information provided, Octank Financial appears to be in a strong financial position and experiencing solid growth. The key indicators that point to this are:

- Revenue increased 

In [16]:
with_reranker_result = evaluate(
    dataset=with_reranker_dataset,
    metrics=metrics,
    llm=llm_for_evaluation,
    embeddings=bedrock_embeddings,
)

with_reranker_result_df = with_reranker_result.to_pandas()

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

In [17]:
with_reranker_result_df

,question,answer,contexts,ground_truth,context_relevancy,context_recall,context_precision,context_entity_recall,answer_correctness
0,What was the primary reason for the increase i...,The primary reason for the increase in net cas...,[** | | | | | Proceeds from issuance of common...,The increase in net cash provided by operating...,0.153846,1.0,1.0,0.000000,0.919630
1,In which year did Octank Financial have the hi...,"According to the search results, Octank Financ...",[| $210 | The consolidated statements of c...,Octank Financial had the highest net cash used...,0.176471,1.0,1.0,0.400000,0.745553
2,What was the primary source of cash inflows fr...,"According to the search results, the primary s...",[. The statement is divided into three main pa...,The primary source of cash inflows from financ...,0.222222,1.0,1.0,0.250000,0.996898
3,"Based on the information provided, what can yo...","Based on the information provided, Octank Fina...","[Net income for 2021 was $1,200 million, an in...","Based on the information provided, Octank Fina...",0.269231,0.2,0.0,0.166667,0.968755


In [18]:
import pandas as pd

# Create the side-by-side DataFrame
comparison_df = pd.DataFrame({
    'question': without_reranker_result_df['question'],
    'without_reranker_answer': without_reranker_result_df['answer'],
    'with_reranker_answer': with_reranker_result_df['answer'],
    'without_reranker_answer_correctness': without_reranker_result_df['answer_correctness'],
    'with_reranker_answer_correctness': with_reranker_result_df['answer_correctness']
})

In [19]:
pd.options.display.max_colwidth = 1000
comparison_df

,question,without_reranker_answer,with_reranker_answer,without_reranker_answer_correctness,with_reranker_answer_correctness
0,What was the primary reason for the increase in net cash provided by operating activities for Octank Financial in 2021?,The primary reason for the increase in net cash provided by operating activities for Octank Financial in 2021 was an increase in net income and favorable changes in operating assets and liabilities.,The primary reason for the increase in net cash provided by operating activities for Octank Financial in 2021 was an increase in net income and favorable changes in operating assets and liabilities.,0.919630,0.919630
1,"In which year did Octank Financial have the highest net cash used in investing activities, and what was the primary reason for this?","According to the search results, Octank Financial had the highest net cash used in investing activities in 2021, at $360 million. The primary reason for this was an increase in purchases of property, plant, and equipment and marketable securities.","According to the search results, Octank Financial had the highest net cash used in investing activities in 2021, at $360 million. The primary reason for this was an increase in purchases of property, plant, and equipment and marketable securities.",0.745553,0.745553
2,What was the primary source of cash inflows from financing activities for Octank Financial in 2021?,"According to the search results, the primary source of cash inflows from financing activities for Octank Financial in 2021 was proceeds from the issuance of long-term debt, which amounted to $500 million.","According to the search results, the primary source of cash inflows from financing activities for Octank Financial in 2021 was an increase in proceeds from issuance of common stock and long-term debt.",0.606073,0.996898
3,"Based on the information provided, what can you infer about Octank Financial's overall financial health and growth prospects?","Based on the information provided, Octank Financial appears to be in strong financial health and have good growth prospects: - The company reported robust revenue growth of 15% in 2021, increasing from $2.5 billion to $2.875 billion. - Net income also grew significantly, from $400 million in 2020 to $500 million in 2021, a 25% increase. - Octank Financial has a strong cash position, with $1 billion in cash and cash equivalents on its balance sheet. - The company has a diverse and talented team of over 1,000 professionals with expertise across various financial services, which suggests it has the capabilities to continue growing its business.","Based on the information provided, Octank Financial appears to be in a strong financial position and experiencing solid growth. The key indicators that point to this are:\n\n- Revenue increased by 11.1% in 2021 compared to 2020, driven by strong performance in the company's core business segments as well as acquisitions of smaller companies. This indicates the company is growing its top-line. - Gross profit increased by 11.1% in 2021, indicating the company is maintaining or improving its profitability despite higher costs for raw materials and labor. - Operating income increased by 6% in 2021, demonstrating the company is effectively managing its operating expenses despite the increases in marketing, advertising, and R&D costs. - Net income grew by 6% in 2021, and earnings per share increased by 6.7%, suggesting the company is generating solid bottom-line growth for its shareholders. Overall, the financial results indicate Octank Financial is a healthy, growing company with strong...",0.366631,0.968755


In [20]:
# Calculate average correctness
without_reranker_avg_correctness = without_reranker_result_df['answer_correctness'].mean()
with_reranker_avg_correctness = with_reranker_result_df['answer_correctness'].mean()

print(f"\nAverage Correctness without Reranker: {without_reranker_avg_correctness:.4f}")
print(f"Average Correctness with Reranker: {with_reranker_avg_correctness:.4f}")


Average Correctness without Reranker: 0.6595
Average Correctness with Reranker: 0.9077


Note: Please note the scores above gives a relative idea on how the reranker model can improve the RAG performance.

Based on the scores, you can review other components of your RAG workflow to further optimize the scores, few recommended options are to review your chunking strategy, prompt instructions, adding more numberOfResults for additional context and so on.